In [8]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 🔹 You change this manually per model
model_dir = r"C:\Users\Gilles\Documents\school\Ma2\Thesis\VSC\IRIS\Own_code\results\OrthoModel_Relu_Nobias"

# 🔹 Where you want the graphs saved (on your PC)
save_dir = os.path.join(model_dir, "graphs")
os.makedirs(save_dir, exist_ok=True)

metrics = [
    "final_train_loss",
    "final_valid_loss",
    "final_train_accuracy",
    "final_valid_accuracy",
    "epochs",
    "epochs_to_final_train",
    "epochs_to_final_valid"
]

# --- LOAD ALL CSV FILES ---
csv_files = [f for f in os.listdir(model_dir) if f.endswith(".csv")]

dfs = []
for file in csv_files:
    dfs.append(pd.read_csv(os.path.join(model_dir, file)))

df = pd.concat(dfs, ignore_index=True)

# Convert hidden_sizes column to a consistent string format
df["hidden_sizes"] = df["hidden_sizes"].astype(str)

# --- SUMMARIZE PER ARCHITECTURE ---
group = df.groupby("hidden_sizes")

summary = {}
for key, df_run in group:
    summary[key] = {}
    for m in metrics:
        summary[key][f"{m}_mean"] = df_run[m].mean()
        summary[key][f"{m}_std"]  = df_run[m].std()

summary_df = pd.DataFrame(summary).T  # Transpose for rows = architectures


# --- PLOT 4×4 GRID WITH HEATMAP AND MARGINAL AVG/STD ---
layer_vals = [1,2,3,4]
neuron_vals = [1,2,3,4]

for metric in metrics:
    fig, ax = plt.subplots(figsize=(6,6))
    ax.set_xticks(range(4))
    ax.set_yticks(range(4))
    ax.set_xticklabels(neuron_vals)
    ax.set_yticklabels(layer_vals)
    ax.set_xlabel("Neurons per layer")
    ax.set_ylabel("Number of layers")
    ax.set_title(metric)
    
    data = np.zeros((len(layer_vals), len(neuron_vals)))
    for i, neur in enumerate(neuron_vals):
        for j, lay in enumerate(layer_vals):
            arch = str([neur]*lay)
            if arch in summary_df.index:
                data[j, i] = summary_df.loc[arch, f"{metric}_mean"]
            else:
                data[j, i] = np.nan

    # Heatmap
    cmap = plt.cm.viridis
    im = ax.imshow(data, origin='lower', cmap=cmap, interpolation='nearest',
                   extent=(-0.5, len(neuron_vals)-0.5, -0.5, len(layer_vals)-0.5))

    # Annotate squares
    for i, neur in enumerate(neuron_vals):
        for j, lay in enumerate(layer_vals):
            arch = str([neur]*lay)
            if arch in summary_df.index:
                mean = summary_df.loc[arch, f"{metric}_mean"]
                std  = summary_df.loc[arch, f"{metric}_std"]
                ax.text(i, j, f"{mean:.4f}\n±{std:.4f}", ha='center', va='center', fontsize=9, color='white')
            ax.add_patch(plt.Rectangle((i-0.5, j-0.5), 1, 1, fill=False))

        # --- Row averages (per layer) ---
    for j, lay in enumerate(layer_vals):
        row_vals = []
        for neur in neuron_vals:
            arch = str([neur]*lay)
            if arch in summary_df.index:
                row_vals.append(summary_df.loc[arch, f"{metric}_mean"])
            else:
                row_vals.append(np.nan)
        row_mean = np.nanmean(row_vals)
        row_std  = np.nanstd(row_vals)
        ax.text(len(neuron_vals)-0.2, j, f"{row_mean:.4f}\n±{row_std:.4f}", va='center', ha='left', fontsize=8, color='black')

    # --- Column averages (per neuron) ---
    for i, neur in enumerate(neuron_vals):
        col_vals = []
        for lay in layer_vals:
            arch = str([neur]*lay)
            if arch in summary_df.index:
                col_vals.append(summary_df.loc[arch, f"{metric}_mean"])
            else:
                col_vals.append(np.nan)
        col_mean = np.nanmean(col_vals)
        col_std  = np.nanstd(col_vals)
        ax.text(i, len(layer_vals)-0.2, f"{col_mean:.4f}\n±{col_std:.4f}", ha='center', va='bottom', fontsize=8, color='black')

    ax.set_xlim(-0.5, len(neuron_vals)-0.5)
    ax.set_ylim(-0.5, len(layer_vals)-0.5)
    plt.tight_layout()
    plt.savefig(os.path.join(save_dir, f"{metric}.png"), dpi=300)
    plt.close(fig)
